In [ ]:
import dtlpy as dl
dl.login()

Create a new Project

In [2]:
try:
    project = dl.projects.get(project_name='Export Storage Driver')
except dl.exceptions.NotFound:
    project = dl.projects.create(project_name='SExport Storage Driver')

org = dl.organizations.get(organization_id=project.org.get('id'))

Create a dataloop integration
This chapter gives as an example setting up aws Access-Key integration for other integration types, please refer to the [developer docs](https://developers.dataloop.ai/tutorials/data_management)

In [3]:
import os
import dotenv

# Create a .env file in the root directory and add the following:
# AWS_KEY=your_access_key_id
# AWS_SECRET=your_secret_access_key
# AWS_REGION=your_region
# AWS_BUCKET=your_bucket_name

dotenv.load_dotenv()

integration = org.integrations.create(
    name='export-storage-driver-s3-demo',
    integrations_type=dl.ExternalStorage.S3,
    options={'key': os.getenv('AWS_KEY'), 'secret': os.getenv('AWS_SECRET')}
)


Create a Storage driver

In [4]:

bucket_name = 'dataloop-export-storage-driver-demo'
driver = project.drivers.create(
    name='s3_driver',
    driver_type=dl.ExternalStorage.S3,
    integration_type=integration.type,
    integration_id=integration.id,
    bucket_name=os.getenv('AWS_BUCKET'),
    allow_external_delete=False, # False is mandatory for this flow
    region=os.getenv('AWS_REGION')
)

Create a System Dataset

In [6]:
try: 
    filters = dl.Filters(resource=dl.FiltersResource.DATASET)
    filters.add(field='name', values=driver.id)
    filters.system_space = True
    dataset = project.datasets.list(filters=filters)[0]
except IndexError:
    dataset = project.datasets.create(dataset_name=driver.id, driver=driver)
    dataset.metadata['system']['scope'] = 'system'
    dataset.update(True)

Upload Data

In [ ]:
dataset.items.upload(local_path = 'assets')

Go to bucket and validate the images uploaded exist.

In [ ]:
dataset.items.delete(filters=dl.Filters())

Go to bucket and validate the images are not deleted.